In [1]:
import pandas as pd
import numpy as np

In [2]:
movies=pd.read_csv("movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies.info() 
movies.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


(9742, 3)

In [4]:
ratings=pd.read_csv("ratings.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
ratings.info() 
ratings.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


(100836, 4)

In [10]:
ratings.movieId.nunique()

9724

In [6]:
utility_matrix = ratings.pivot(index='movieId', columns='userId', values='rating').fillna(0)
utility_matrix

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
def adjusted_cosine_sim_vectorized(utility_matrix,movie_list):
    """
    Returns Construct item-item (Adjusted Cosine) similarity matrix .

            Parameters:
                utility matrix : Movies(item) as index and users as column --> shape(num_movies, number_users)

            Returns:
                similarity matrix
    """
    norm_utility_matrix = utility_matrix - np.mean(utility_matrix ,axis=1).reshape(utility_matrix.shape[0],-1)
    # calculate norm for all item vectors
    all_item_norm = np.linalg.norm(norm_utility_matrix ,axis=1).reshape(1, norm_utility_matrix.shape[0])
    similarity_matrix = (norm_utility_matrix @ norm_utility_matrix.T) / (all_item_norm.T * all_item_norm)
    similarity_matrix = pd.DataFrame(data=np.round(similarity_matrix,2), columns=movie_list, index=movie_list)
    return similarity_matrix

In [8]:
def similar_items_by_id(movie_id):
    similarity_matrix = adjusted_cosine_sim_vectorized(np.array(utility_matrix),movie_list=list(utility_matrix.index))
    sorted=similarity_matrix[movie_id].sort_values(ascending=False)[1:]
    ids =sorted.index.tolist()
    filtered_movies = movies.loc[movies["movieId"].isin(ids)]
    sorted_df = filtered_movies.sort_values(by='movieId', key=lambda x: x.map({v: i for i, v in enumerate(ids)}))
    sorted_df["similarity score"]=sorted.values
    return sorted_df

In [9]:
similar_items_by_id(3)

,movieId,title,genres,similarity score
2581,3450,Grumpy Old Men (1993),Comedy,0.42
607,762,Striptease (1996),Comedy|Crime,0.41
622,788,"Nutty Professor, The (1996)",Comedy|Fantasy|Romance|Sci-Fi,0.39
475,542,Son in Law (1993),Comedy|Drama|Romance,0.38
58,65,Bio-Dome (1996),Comedy,0.37
...,...,...,...,...
8303,106696,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,-0.06
7802,92259,Intouchables (2011),Comedy|Drama,-0.07
6165,44555,"Lives of Others, The (Das leben der Anderen) (...",Drama|Romance|Thriller,-0.07
3622,4973,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance,-0.07


In [11]:
def similar_item_by_name(movie_name):
    movie_id = movies.loc[movies['title'] =='Grumpier Old Men (1995)', 'movieId'].values[0]
    similarity_matrix = adjusted_cosine_sim_vectorized(np.array(utility_matrix),movie_list=list(utility_matrix.index))
    sorted=similarity_matrix[movie_id].sort_values(ascending=False)[1:]
    ids =sorted.index.tolist()
    filtered_movies = movies.loc[movies["movieId"].isin(ids)]
    sorted_df = filtered_movies.sort_values(by='movieId', key=lambda x: x.map({v: i for i, v in enumerate(ids)}))
    sorted_df["similarity score"]=sorted.values
    return sorted_df

In [12]:
similar_item_by_name('Grumpier Old Men (1995)')

,movieId,title,genres,similarity score
2581,3450,Grumpy Old Men (1993),Comedy,0.42
607,762,Striptease (1996),Comedy|Crime,0.41
622,788,"Nutty Professor, The (1996)",Comedy|Fantasy|Romance|Sci-Fi,0.39
475,542,Son in Law (1993),Comedy|Drama|Romance,0.38
58,65,Bio-Dome (1996),Comedy,0.37
...,...,...,...,...
8303,106696,Frozen (2013),Adventure|Animation|Comedy|Fantasy|Musical|Rom...,-0.06
7802,92259,Intouchables (2011),Comedy|Drama,-0.07
6165,44555,"Lives of Others, The (Das leben der Anderen) (...",Drama|Romance|Thriller,-0.07
3622,4973,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance,-0.07
